In [1]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime
import os

entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

feast_repo_path = "{}/{}/feature_repo/".format(os.getenv('HOME'), os.getenv('FEATYRE_REPO_PATH'))

store = FeatureStore(repo_path=feast_repo_path)

training_df = store.get_historical_features(
    entity_df=entity_df,
    features = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

print(training_df.head())

   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.557606  0.126050              217
1       1002 2021-04-12 08:12:10+00:00   0.783325  0.025297              567
2       1003 2021-04-12 16:40:26+00:00   0.101772  0.891472              361
3       1004 2021-04-12 15:01:12+00:00   0.301313  0.454685              597


In [2]:
from feast import FeatureStore
import os

feast_repo_path = "{}/{}/feature_repo/".format(os.getenv('HOME'), os.getenv('FEATYRE_REPO_PATH'))
#store = FeatureStore(repo_path=feast_repo_path)

In [3]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [4]:
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [5]:
class FeastPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
            ],
            entity_rows=[{"driver_id": driver_id}],
        ).to_dict()
        # print("driver_id: {}".format(driver_id))
        # print("features: {}".format(feature_vector))
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [6]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [7]:
print(prompt_template.format(driver_id=1001))

driver_id: 1001
features: {'driver_id': [1001], 'conv_rate': [0.6366674900054932], 'avg_daily_trips': [424], 'acc_rate': [0.6976584792137146]}
Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: 0.6366674900054932
Acceptance rate: 0.6976584792137146
Average Daily Trips: 424

Your response:


In [9]:
# Into LLM chain
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt_template)

chain.run(1004)

driver_id: 1004
features: {'driver_id': [1004], 'conv_rate': [0.5550095438957214], 'avg_daily_trips': [913], 'acc_rate': [0.49236470460891724]}


"Dear Driver,\n\nI hope this message finds you well. I wanted to provide you with an update on your current statistics. Your conversation rate is an impressive 0.5550095438957214, which is fantastic! Your ability to engage in meaningful conversations with passengers is truly commendable. Keep up the excellent work!\n\nAdditionally, your acceptance rate stands at 0.49236470460891724. While there is always room for improvement, I appreciate your efforts in accepting a significant number of trip requests. Remember, every trip counts towards building a positive relationship with our passengers.\n\nOn another note, your average daily trips amount to 913. That's quite an impressive number! Your dedication and commitment to providing reliable transportation services are truly appreciated.\n\nIf I may add a touch of humor, did you know that chickens are excellent drivers? They always stay in their lanes because they have eggs-cellent focus! So even if your conversation rate is not above 0.5, r